In [1]:
import pandas as pd

## Fields to be acquired

orig_food_common_name

orig_food_scientific_name

orig_food_part

orig_source_id

orig_source_name

orig_content

### Toy dataset of same shape, to ensure the proper reshaping transformations will be done 

In [2]:
food_id               = [1, 1, 1, 2, 2, 2, 3, 3, 3, 3.0]
orig_food_common_name = ['Carrot', 'Carrot', 'Carrot', 'Kale',  'Kale', 'Kale', 'Beef', 'Beef', 'Beef', 'Beef (cow)']
orig_food_part        = ['leaf',   'root',   'root', 'stalk', 'leaf', 'root', 'leg',  'leg',  'leaf', 'hoof']
orig_source_name      = ['FAT', 'FAT', 'fiber', 'protein', 'sodium', 'protein', 'sodium', 'FAT', 'sodium', 'protein']
orig_content          = [200, 1300, 500, 1.5, 2.5, 1, 15, 10, 5, 7]

df = pd.DataFrame({'food_id':food_id,'orig_food_common_name': orig_food_common_name, 'orig_food_part': orig_food_part, 'orig_source_name': orig_source_name, 'orig_content':orig_content})

In [3]:
df['compound_in_part'] = df['orig_source_name'] + '_in_' +  df['orig_food_part'] 


In [4]:
df

,food_id,orig_food_common_name,orig_food_part,orig_source_name,orig_content,compound_in_part
0,1.0,Carrot,leaf,FAT,200.0,FAT_in_leaf
1,1.0,Carrot,root,FAT,1300.0,FAT_in_root
2,1.0,Carrot,root,fiber,500.0,fiber_in_root
3,2.0,Kale,stalk,protein,1.5,protein_in_stalk
4,2.0,Kale,leaf,sodium,2.5,sodium_in_leaf
5,2.0,Kale,root,protein,1.0,protein_in_root
6,3.0,Beef,leg,sodium,15.0,sodium_in_leg
7,3.0,Beef,leg,FAT,10.0,FAT_in_leg
8,3.0,Beef,leaf,sodium,5.0,sodium_in_leaf
9,3.0,Beef (cow),hoof,protein,7.0,protein_in_hoof


In [6]:
cols = ['orig_food_common_name', 'compound_in_part','orig_content']
df[cols]

,orig_food_common_name,compound_in_part,orig_content
0,Carrot,FAT_in_leaf,200.0
1,Carrot,FAT_in_root,1300.0
2,Carrot,fiber_in_root,500.0
3,Kale,protein_in_stalk,1.5
4,Kale,sodium_in_leaf,2.5
5,Kale,protein_in_root,1.0
6,Beef,sodium_in_leg,15.0
7,Beef,FAT_in_leg,10.0
8,Beef,sodium_in_leaf,5.0
9,Beef (cow),protein_in_hoof,7.0


In [7]:
spread1_df_concentration = df[cols].groupby(['orig_food_common_name', 'compound_in_part']).mean().squeeze().unstack()

spread1_df_concentration 

compound_in_part,FAT_in_leaf,FAT_in_leg,FAT_in_root,fiber_in_root,protein_in_hoof,protein_in_root,protein_in_stalk,sodium_in_leaf,sodium_in_leg
orig_food_common_name,,,,,,,,,
Beef,NaN,10.0,NaN,NaN,NaN,NaN,NaN,5.0,15.0
Beef (cow),NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN
Carrot,200.0,NaN,1300.0,500.0,NaN,NaN,NaN,NaN,NaN
Kale,NaN,NaN,NaN,NaN,NaN,1.0,1.5,2.5,NaN


...now that we know the restructuing will work, we can...

## Read in the actual 5-million row dataset

In [23]:
content_df = pd.read_csv('food_sources/foodb_2020_04_07_csv/Content.csv')


/opt/anaconda3/envs/metis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,5,6,7,8,9,13,14,21,22,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [24]:
df = content_df

In [25]:
df['orig_food_common_name']=df['orig_food_common_name'].str.lower()
df['food_id']=df['food_id'].astype(int)

In [26]:
df.shape

(5145532, 26)

In [27]:
food_id_df = df.loc[:,['food_id', 'orig_food_common_name']]

In [28]:
food_id_df

,food_id,orig_food_common_name
0,4,kiwi
1,6,onion
2,6,onion
3,9,chives
4,11,cashew
...,...,...
5145527,555,endogenous compounds from human (hmdb)
5145528,590,endogenous compounds from human (hmdb)
5145529,595,endogenous compounds from human (hmdb)
5145530,611,endogenous compounds from human (hmdb)


In [29]:
food_id_df = food_id_df.groupby(['orig_food_common_name']).max().reset_index()
food_id_df

,orig_food_common_name,food_id
0,(capsicum anuum),40
1,(cucumis melo),64
2,(ipomoea batatas),92
3,(lactuca sativa var. crispa),95
4,(medicago sativa),287
...,...,...
9448,"yogurt, vanilla or lemon flavor, nonfat milk, ...",634
9449,"yogurt, vanilla, low fat, 11 grams protein per...",634
9450,"yogurt, vanilla, low fat, 11 grams protein per...",634
9451,"zucchini, raw",908


In [30]:
food_id_and_name = food_id_df.groupby(['food_id']).agg(lambda x:x.value_counts().index[0])

In [31]:
food_id_and_name

,orig_food_common_name
food_id,
1,"angelica, fresh"
2,"savoy cabbage, raw"
3,"silver linden, dried"
4,"kiwi, pure juice"
5,"onions, yellow, sauteed"
...,...
1021,"tea, herb, other than chamomile, brewed"
1022,"fish oil, hydrogenated"
1023,"taco shells, baked"


In [32]:
df['compound_in_part'] = df['orig_source_name'] + '_in_' +  df['orig_food_part'] 

In [33]:
df['food_id'].nunique()

987

In [34]:
cols = ['orig_food_common_name', 'compound_in_part','orig_content']
df[cols]

,orig_food_common_name,compound_in_part,orig_content
0,kiwi,FAT_in_Fruit,1955.00
1,onion,FAT_in_Bulb,1853.95
2,onion,FAT_in_Leaf,4150.00
3,chives,FAT_in_Leaf,3900.00
4,cashew,FAT_in_Fruit,2500.00
...,...,...,...
5145527,endogenous compounds from human (hmdb),NaN,NaN
5145528,endogenous compounds from human (hmdb),NaN,NaN
5145529,endogenous compounds from human (hmdb),NaN,NaN
5145530,endogenous compounds from human (hmdb),NaN,NaN


In [35]:
spread_df_concentration = df[cols].groupby(['orig_food_common_name', 'compound_in_part']).mean().squeeze().unstack()

spread_df_concentration.head()

compound_in_part,(E)-2-HEPTENAL_in_Calyx,(E)-2-HEXENAL_in_Calyx,(Z)-3-HEXENOL_in_Calyx,"1,4-CINEOLE_in_Calyx","1,8-CINEOLE_in_Calyx",1-HEXANOL_in_Calyx,"2,2-DIMETHYL-HEXANAL_in_Calyx",2-ETHYL-FURAN_in_Calyx,2-PENTYL-FURAN_in_Calyx,3-METHYL-1-BUTANOL|ISOAMYL-ALCOHOL_in_Fruit,...,VIT-E_in_Sprout Seedling,ALLANTOIC-ACID_in_Sprout Seedling,HOMOGENTISINIC-ACID_in_Sprout Seedling,PIPECOLIC-ACID_in_Stem,NEOGLUCOBRASSICIN_in_Seed,TETRACOSANOIC-ACID_in_Seed Oil,DIOSCIN_in_Root,ADENINE_in_Tuber,PHYTIN_in_Tuber,SUGARS_in_Tuber
orig_food_common_name,,,,,,,,,,,,,,,,,,,,,
acedera de guinea (sp.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aceituna dulce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
achnef (berber; bou),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
adzuki bean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ajonjoli (sp.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
spread_df_concentration.head(50)

compound_in_part,(E)-2-HEPTENAL_in_Calyx,(E)-2-HEXENAL_in_Calyx,(Z)-3-HEXENOL_in_Calyx,"1,4-CINEOLE_in_Calyx","1,8-CINEOLE_in_Calyx",1-HEXANOL_in_Calyx,"2,2-DIMETHYL-HEXANAL_in_Calyx",2-ETHYL-FURAN_in_Calyx,2-PENTYL-FURAN_in_Calyx,3-METHYL-1-BUTANOL|ISOAMYL-ALCOHOL_in_Fruit,...,VIT-E_in_Sprout Seedling,ALLANTOIC-ACID_in_Sprout Seedling,HOMOGENTISINIC-ACID_in_Sprout Seedling,PIPECOLIC-ACID_in_Stem,NEOGLUCOBRASSICIN_in_Seed,TETRACOSANOIC-ACID_in_Seed Oil,DIOSCIN_in_Root,ADENINE_in_Tuber,PHYTIN_in_Tuber,SUGARS_in_Tuber
orig_food_common_name,,,,,,,,,,,,,,,,,,,,,
acedera de guinea (sp.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aceituna dulce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
achnef (berber; bou),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
adzuki bean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ajonjoli (sp.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
alholva (sp.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
allspice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
almond,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
american cranberry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
spread_df_concentration.shape

(238, 15597)

In [49]:
foods_recomb_df = food_id_and_name.join(spread_df_concentration, on = 'orig_food_common_name', how='left')

In [50]:
foods_recomb_df

,orig_food_common_name,(E)-2-HEPTENAL_in_Calyx,(E)-2-HEXENAL_in_Calyx,(Z)-3-HEXENOL_in_Calyx,"1,4-CINEOLE_in_Calyx","1,8-CINEOLE_in_Calyx",1-HEXANOL_in_Calyx,"2,2-DIMETHYL-HEXANAL_in_Calyx",2-ETHYL-FURAN_in_Calyx,2-PENTYL-FURAN_in_Calyx,...,VIT-E_in_Sprout Seedling,ALLANTOIC-ACID_in_Sprout Seedling,HOMOGENTISINIC-ACID_in_Sprout Seedling,PIPECOLIC-ACID_in_Stem,NEOGLUCOBRASSICIN_in_Seed,TETRACOSANOIC-ACID_in_Seed Oil,DIOSCIN_in_Root,ADENINE_in_Tuber,PHYTIN_in_Tuber,SUGARS_in_Tuber
food_id,,,,,,,,,,,,,,,,,,,,,
1,"angelica, fresh",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"savoy cabbage, raw",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"silver linden, dried",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"kiwi, pure juice",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"onions, yellow, sauteed",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,"tea, herb, other than chamomile, brewed",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1022,"fish oil, hydrogenated",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1023,"taco shells, baked",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
foods_recomb_df.fillna(0)

,orig_food_common_name,(E)-2-HEPTENAL_in_Calyx,(E)-2-HEXENAL_in_Calyx,(Z)-3-HEXENOL_in_Calyx,"1,4-CINEOLE_in_Calyx","1,8-CINEOLE_in_Calyx",1-HEXANOL_in_Calyx,"2,2-DIMETHYL-HEXANAL_in_Calyx",2-ETHYL-FURAN_in_Calyx,2-PENTYL-FURAN_in_Calyx,...,VIT-E_in_Sprout Seedling,ALLANTOIC-ACID_in_Sprout Seedling,HOMOGENTISINIC-ACID_in_Sprout Seedling,PIPECOLIC-ACID_in_Stem,NEOGLUCOBRASSICIN_in_Seed,TETRACOSANOIC-ACID_in_Seed Oil,DIOSCIN_in_Root,ADENINE_in_Tuber,PHYTIN_in_Tuber,SUGARS_in_Tuber
food_id,,,,,,,,,,,,,,,,,,,,,
1,"angelica, fresh",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"savoy cabbage, raw",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"silver linden, dried",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"kiwi, pure juice",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"onions, yellow, sauteed",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,"tea, herb, other than chamomile, brewed",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1022,"fish oil, hydrogenated",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1023,"taco shells, baked",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 900 foods with 15K+ continuous features ready for analysis

In [52]:

with open("data/" + "foods_recomb_df.pickle", 'wb') as to_write:
    pickle.dump(foods_recomb_df, to_write)

## All scratch work below this line. It can be ignored